## Introduction
This notebook is a tutorial on using VF-iDCA to solve the hyperparameter selection problem. And we take the following w-Lasso problem for example,
$$\begin{aligned}
\min_{\lambda}\ & 
\|X_{\text{val}}\hat{w} - y_{\text{val}}\|^2 \\ 
\text{where } & 
\hat{w} \in \mathop{\arg\min}_{w} 
\|X_{\text{tr}}w - y_{\text{tr}}\|^2 + \sum_{i=1}^p \lambda_i |w_i|
\end{aligned}$$
With hyperparameter decoupling, we are solving the following problem
$$\begin{aligned}
& \min_{\lambda}\ 
\|X_{\text{val}}\hat{w} - y_{\text{val}}\|^2 \\ 
& \begin{aligned}
\text{where } \hat{w} \in \mathop{\arg\min}_{w}\ &
\|X_{\text{tr}}w - y_{\text{tr}}\|^2 \\
\text{s.t. } & |w_i| \le r_i, \quad i = 1,\dots, p
\end{aligned}
\end{aligned}$$

## Structure
Here is the structure of this tutorial:
- How to construct and solve a subproblem with `cvxpy`
- How to use VF-iDCA
- Some advice for acceleration

## Solving w-Lasso problem with `cvxpy`
We firstly generate some synetic data for illustration.

In [3]:
# Generate Data
import numpy as np

np.random.seed(42) # for reproduction

n, p = 600, 100
beta_nonzero = np.array([1,2,3,4,5])
beta_real = np.concatenate([beta_nonzero, np.zeros(p - len(beta_nonzero))])
X = np.random.randn(n, p)
y_true = X @ beta_real

# add noise
snr = 2
epsilon = np.random.randn(n)
SNR_factor = snr / np.linalg.norm(y_true) * np.linalg.norm(epsilon)
y = y_true + 1.0 / SNR_factor * epsilon

# split data
nTr, nVal = 100, 100
XTr, XVal, XTest = X[0:nTr], X[nTr:nTr + nVal], X[nTr + nVal:]
yTr, yVal, yTest = y[0:nTr], y[nTr:nTr + nVal], y[nTr + nVal:]

## define err function
def ls_err(X, y, w):
    return np.sum(np.square(X @ w - y))/len(y)

def err_report(w):
    print("%20s %8.3f" % ("train err:", ls_err(XTr, yTr, w)))
    print("%20s %8.3f" % ("validate err:", ls_err(XVal, yVal, w)))
    print("%20s %8.3f" % ("test err:", ls_err(XTest, yTest, w)))

The original wlasso problem could be solving as 

In [2]:
import cvxpy as cp 

## define the wlasso problem
w = cp.Variable(p) # define the variables
lam = cp.Parameter(p, nonneg=True) # define the regularization parameters, nonneg=True means its value will not be negtive
wlasso_obj = cp.sum_squares(XTr @ w - yTr) + lam @ cp.abs(w) # define the objective of w-lasso problem, to check the supported function, one should refer to https://www.cvxpy.org/tutorial/functions/index.html#functions
wlasso = cp.Problem(cp.Minimize(wlasso_obj)) # define the problem

## solve a wlasso problem
lam.value = np.ones(p) # clarify the value of parameters, here we use [1,1,..,1]
wlasso.solve(solver = cp.ECOS) # use ECOS solver to solve the problem
print("optimal objective is %.3f" % (wlasso.value))
print("first 3 values of optimal w are", w.value[:3])
err_report(w.value)

NameError: name 'p' is not defined

The decoupled wlasso problem could be solving as 

In [102]:
## define the wlasso problem
w = cp.Variable(p) # define the variables
r = cp.Parameter(p, nonneg=True) # define the regularization parameters
wlasso_obj = cp.sum_squares(XTr @ w - yTr) 
cons = [w[i] <= r[i] for i in range(p)]
wlasso = cp.Problem(cp.Minimize(wlasso_obj), cons) # define the problem

## solve a wlasso problem
r.value = np.ones(p) # clarify the value of parameters, here we use [1,1,..,1]
wlasso.solve(solver = cp.ECOS) # use ECOS solver to solve the problem
print("optimal objective is %.3f" % (wlasso.value))
print("first 3 values of optimal w are", w.value[:3])
print("first value of the corresponding lam are", cons[0].dual_value)
err_report(w.value)

optimal objective is 820.304
first 3 values of optimal w are [0.7082588  0.99999997 1.        ]
first value of the corresponding lam are 3.137914137504083e-06
          train err:    8.203
       validate err:  107.124
           test err:  135.303


## Use VF-iDCA to select hyperparamter
Before we apply the VF-iDCA method, we would like to clarify how to solve the following approximated problem
$$\begin{aligned}
\min_{w, r} \phi_k(w, r) = & \|X_{\text{Val}}w - y\|^2 + \frac{\rho}2 (\|w - w^k\|^2 + \|r - r^k\|^2)\\  & + \alpha_k \max\{0,V_k(w, r), \max_{i=1,\dots,p}|w_i| - r_i\}
\end{aligned}$$
where
$$\begin{aligned}
V_k(w, r) = & \|X_{\text{Tr}}w - y\|^2 - \|X_{\text{Tr}}\hat{w}^k - y\|^2
+ \langle \gamma^k, r - r^k \rangle \\
= & \|X_{\text{Tr}}w - y\|^2 - l(\hat{w}^k)
+ \langle \gamma^k, r - r^k \rangle
\end{aligned}$$

In [103]:
## define the approximated problem
rho = 1. 
wU = cp.Variable(p) # define the variables
rU = cp.Variable(p) # define the regularization parameters
fL = cp.Parameter()
wk = cp.Parameter(p)
rk = cp.Parameter(p, nonneg=True)
gamma = cp.Parameter(p)
alpha = cp.Parameter(nonneg=True)

ls_val = cp.sum_squares(XVal @ wU - yVal)
prox = cp.sum_squares(wU - wk) + cp.sum_squares(rU - rk)
Vk = cp.sum_squares(XTr @ wU - yTr) - fL + gamma @ (r - rk)
violation = cp.maximum(*([0, Vk] + [cp.abs(wU[i]) - rU[i] for i in range(p)]))

phi_k = ls_val + rho/2 * prox + alpha * violation
bi_cons = [rU >= 0]

dc_app = cp.Problem(cp.Minimize(phi_k), bi_cons)

fL.value = wlasso.value
wk.value = np.zeros(p)
rk.value = np.ones(p)
gamma.value = np.array([float(cons[i].dual_value) for i in range(p)])
alpha.value = 1.

dc_app.solve(solver = cp.ECOS)
print("optimal objective is %.3f" % (dc_app.value))
print("first 3 values of optimal w are", wU.value[:3])
print("first 3 values of optimal r are", rU.value[:3])
err_report(wU.value)

d:\_Tools\anaconda3\envs\cvxpy\lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:167: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)


optimal objective is 255.101
first 3 values of optimal w are [0.31829751 1.8252355  2.91677977]
first 3 values of optimal r are [0.99998267 0.9999788  0.99996856]
          train err:    8.241
       validate err:    2.146
           test err:   24.770


In [43]:
fL.value 

820.303557332848

To use VF-iDCA more easily, we need define both lower-level problem and approximated problem in the following way.

In [4]:
class DC_lower():
    def __init__(self):
        self.wL = cp.Variable(p)
        self.rL = cp.Parameter(p, nonneg=True)
        LSTr = cp.sum_squares(XTr @ self.wL - yTr) 
        self.cons = [self.wL[i] <= self.rL[i] for i in range(p)]
        self.dc_lower = cp.Problem(cp.Minimize(LSTr), self.cons)

    def solve(self, r):
        self.rL.value = r
        result = self.dc_lower.solve(solver = cp.ECOS)
        return result

    def dual_value(self):
        return np.array([float(self.cons[i].dual_value) for i in range(p)])

class DC_approximated():
    def __init__(self):
        self.rho = 1.
        self.delta = 5.
        self.c_alpha = 1. 

        self.wU = cp.Variable(p)
        self.rU = cp.Variable(p)
        self.fL = cp.Parameter()
        self.wk = cp.Parameter(p)
        self.rk = cp.Parameter(p, nonneg=True)
        self.gamma = cp.Parameter(p)
        self.alpha = cp.Parameter(nonneg=True)

        self.alpha.value = 1.

        LSVal = cp.sum_squares(XVal @ self.wU - yVal)
        prox = cp.sum_squares(self.wU - self.wk) + cp.sum_squares(self.rU - self.rk)
        Vk = cp.sum_squares(XTr @ self.wU - yTr) - self.fL + self.gamma @ (self.rU - self.rk)
        self.violation = cp.maximum(*([0, Vk] + [cp.abs(self.wU[i]) - self.rU[i] for i in range(p)]))

        phi_k = LSVal + self.rho/2 * prox + self.alpha * self.violation
        bi_cons = [self.rU >= 0]

        self.dc_app = cp.Problem(cp.Minimize(phi_k), bi_cons)
    
    def solve(self):
        self.dc_app.solve(solver = cp.ECOS)
        return self.dc_app.value, self.wU.value, np.maximum(0, self.rU.value)
    
    def clare_variable(self, w, r):
        self.wk.value = w 
        self.rk.value = r
    
    def clare_V(self, fL, gamma):
        self.fL.value = fL
        self.gamma.value = gamma

# Test the function
dclower = DC_lower()
dcapp = DC_approximated()
print(dclower.solve(np.ones(p)))
fL = dclower.solve(np.ones(p))
gamma = dclower.dual_value()
dcapp.clare_variable(np.zeros(p), np.ones(p))
dcapp.clare_V(fL, gamma)
print(dcapp.solve()[0])


820.303557332848


d:\_Tools\anaconda3\envs\cvxpy\lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:167: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)


130.04023506468948


In [6]:
def iteration_err(w, r, wp, rp):
    return np.sqrt(
        np.sum(np.square(w - wp)) + np.sum(np.square(r - rp))
    ) / np.sqrt(
        1 + np.sum(np.square(w)) + np.sum(np.square(r))
    )

def VF_iDCA(dclower, dcapp, DC_Setting = dict()):
    MAX_ITERATION = DC_Setting["MAX_ITERATION"] if "MAX_ITERATION" in DC_Setting.keys() else 10
    TOL = DC_Setting["TOL"] if "TOL" in DC_Setting.keys() else 1e-1
    dcapp.alpha.value = DC_Setting["alpha"] if "alpha" in DC_Setting.keys() else 1.
    dcapp.c_alpha = DC_Setting["c_alpha"] if "c_alpha" in DC_Setting.keys() else 1.
    dcapp.delta = DC_Setting["delta"] if "delta" in DC_Setting.keys() else 5. 
    r = DC_Setting["initial_r"] if "initial_r" in DC_Setting.keys() else np.ones(dcapp.rU.shape)
    w = DC_Setting["initial_w"] if "initial_w" in DC_Setting.keys() else np.zeros(dcapp.wU.shape)

    for _ in range(MAX_ITERATION):
        fL = dclower.solve(r)
        gamma = dclower.dual_value()
        dcapp.clare_variable(w, r)
        dcapp.clare_V(fL, gamma)
        _, wp, rp = dcapp.solve()

        err = iteration_err(w, r, wp, rp) 
        penalty = dcapp.violation.value

        if err < TOL and penalty < TOL:
            break

        if err * dcapp.alpha.value <= dcapp.c_alpha * min( 1., dcapp.alpha.value * dcapp.violation.value ):
            dcapp.alpha.value = dcapp.alpha.value + dcapp.delta

        w, r = wp, rp

    return wp, rp  
        
dclower = DC_lower()
dcapp = DC_approximated()
wp, rp = VF_iDCA(dclower, dcapp, dict())

In [7]:
err_report(wp)

          train err:   24.317
       validate err:    0.643
           test err:   37.701


## How to accelerate VF-iDCA
However, such a programming works slowly since the parameterized problem is not DPP. 

Because the problem is not DPP, subsequent solves will not be faster than the first one. 

Documentation about DPP is at 
https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming

The main problem is that DPP don't allow the multiplication between parameters (even when the signs are given), so you need to take good care on `alpha*violation`.

For more details on acceleration, please refer to the documentation and the example codes of other problems.

Any question about programming, please contact 11930905@mail.sustech.edu.cn.